In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataCleaningConfig:
    root_dir: Path
    data_path: Path
    target_column: str
    object_columns: list

In [12]:
os.chdir("../")

In [13]:
%pwd


'c:\\Users\\dzmitry\\Desktop\\data_science_project'

In [10]:
os.chdir("data_science_project")

FileNotFoundError: [WinError 2] Nie można odnaleźć określonego pliku: 'data_science_project'

In [9]:
import os

In [14]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_cleaning_config(self) -> DataCleaningConfig:
        config = self.config.data_cleaning
        target_column = self.schema.TARGET_COLUMN
        object_columns = self.schema.OBJECT_COLUMNS

        create_directories([config.root_dir])

        data_cleaning_config = DataCleaningConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target_column=target_column.name,
            object_columns=object_columns
        )

        return data_cleaning_config

In [17]:
import pandas as pd
import numpy as np
import os
from src.datascience import logger

In [ ]:
class DataCleaning:
    def __init__(self, config: DataCleaningConfig):
        self.config = config
    
    def clean_data(self):
        data = pd.read_csv(self.config.data_path)
        logger.info(data.shape)
        data=pd.get_dummies(data,columns=self.config.object_columns,drop_first=True)
        data[self.config.target_column]=np.where(data[self.config.target_column]=="Y",1,0)
        data=data.dropna()
        
        data.to_csv(os.path.join(self.config.root_dir, "credit_risk_dataset.csv"),index = False)
        logger.info("Data cleaning completed successfully")
        logger.info(data.shape)
        
        print(data.shape)

In [21]:
try:   
    config=ConfigurationManager()
    config_cleaning=config.get_data_cleaning_config()
    data_cleaning=DataCleaning(config=config_cleaning)
    data_cleaning.clean_data()
except Exception as e:
    raise e

[2026-01-04 17:13:41,772: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-04 17:13:41,779: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-04 17:13:41,784: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-04 17:13:41,787: INFO: common: created directory at: artifacts]
[2026-01-04 17:13:41,790: INFO: common: created directory at: artifacts/data_cleaning]
[2026-01-04 17:13:42,055: INFO: 227890570: Data cleaning completed successfully]
[2026-01-04 17:13:42,055: INFO: 227890570: (28638, 23)]
(28638, 23)


In [34]:
%pwd
df=pd.read_csv(r'c:\\Users\\dzmitry\\Desktop\\data_science_project\\artifacts\\data_cleaning\\credit_risk_dataset.csv')

In [22]:
config_cleaning

DataCleaningConfig(root_dir='artifacts/data_cleaning', data_path='artifacts/data_ingestion/credit_risk_dataset.csv', target_column='cb_person_default_on_file', object_columns=BoxList(['person_home_ownership', 'loan_intent', 'loan_grade']))